# 06. Integration of RNA-seq and DNA methylation data

Simultaneous profiling of RNA-seq and DNA methylation from cells broadens our understanding about the interaction between transcriptomics and epigenomics. In this tutorial, we introduce how to integrate scRNA-seq and scBS-seq (DNA methylation) using scMaui by using the mouse embryo scNMT-seq data set from [GSE121708](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE121708).

### Data Preprocessing
The data set provides a detailed pipeline of data processing on their [Github repo](https://github.com/rargelaguet/scnmt_gastrulation). RNA-seq count matrix can be downloaded [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE121650). However, scBS-seq data is not provided as a matrix but in .tsv file with BED format.

In [19]:
import pandas as pd
pd.read_csv("../../GSE121650_scRNA_methyl/met/GSM3443369_E4.5-5.5_new_Plate1_A02_met.tsv", 
            sep="\t", nrows=5)

,chr,pos,met_reads,nonmet_reads,rate,end
0,chr1,3003379,1,0,1,3003380
1,chr1,3008546,1,0,1,3008547
2,chr1,3017509,2,1,1,3017510
3,chr1,3017624,1,0,1,3017625
4,chr1,3020144,2,0,1,3020145


Therefore, we used an R package [Methrix](https://www.bioconductor.org/packages/release/bioc/html/methrix.html) to convert the BED-formatted files into a cell x region matrix containing methylation beta-values (but it only has 0 or 1 since the data is from single-cells) following [the tutorial](https://www.bioconductor.org/packages/release/bioc/vignettes/methrix/inst/doc/methrix.html#Reading_bedgraph_files). After reading the .tsv files with ```methrix::read_bedgraphs```, you can extract the beta-value matrix using ```methrix::get_matrix```. 

Once you have a matrix, you can follow the [AnnData tutorial](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html) to create an anndata object with the data. **Please note that GSE121708 also provides quality control (QC) results. We only used data which passed the QC**

### Load Data 
We have 939 cells with 10,000 features for RNA-seq and 1,174 features for BS-seq. We selected top 10,000 highly variable genes for the RNA-seq modality, whereas we chose promoter and enhancer regions for the BS-seq modality. Please find the details about promoter and enhancer regions in the [Methods](https://www.biorxiv.org/content/10.1101/2023.01.18.524506v1) section.

In [20]:
from scmaui.utils import init_model_params
from scmaui.data import load_data, SCDataset
from scmaui.ensembles import EnsembleVAE
import os 
import warnings
warnings.filterwarnings('ignore') # to remove warnings

adatas = load_data(["../data/scNMT-seq/rna.h5ad", 
                    "../data/scNMT-seq/methyl.h5ad"], names=['gex', 'methyl'])

In [21]:
adatas["input"]

[AnnData object with n_obs × n_vars = 939 × 10000
     obs: 'id_rna', 'id_met', 'id_acc', 'embryo', 'plate', 'stage', 'lineage10x', 'lineage10x_2'
     var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'view'
     uns: 'embryo_colors', 'hvg', 'lineage10x_2_colors', 'neighbors', 'stage_colors', 'umap', 'view'
     obsm: 'X_pca', 'X_umap', 'mask'
     obsp: 'connectivities', 'distances',
 AnnData object with n_obs × n_vars = 939 × 1174
     obs: 'id_rna', 'id_met', 'id_acc', 'embryo', 'plate', 'stage', 'lineage10x', 'lineage10x_2'
     var: 'view'
     uns: 'view'
     obsm: 'mask']

scBS-seq data has much higher sparsity than other single-cell omics assays. **Missing values can be given as NaN**, so that scMaui does not calculate reconstruction losses using the missing values. 

In [28]:
adatas["input"][1].to_df().iloc[:3,212:216]

,chr7_17201476-17205476,chr7_17148280-17152280,chr7_16673706-16677706,chr7_16257541-16261541
sample,,,,
E4.5-5.5_new_Plate3_E09,0.0,NaN,NaN,NaN
E4.5-5.5_new_Plate3_H09,NaN,NaN,NaN,NaN
E4.5-5.5_new_Plate4_F01,NaN,NaN,NaN,NaN


### Run scMaui

Now we run scMaui to integrate scRNA-seq and scBS-seq modalities. Details about running scMaui is described in [Tutorial 02](https://github.com/BIMSBbioinfo/scmaui-experiments/blob/5347ce3673602dd8d0ff89760ebb212d8707aa38/tutorials/02.%20Run%20scMaui%20on%20single-cell%20multiomics%20dataset.ipynb). **Please, note that we use binary loss for the scBS-seq modality considering the binary values in the modality**. Embryo information is assigned as a batch effect factor. Downstream analysis can be done following [Tutorial 03](https://github.com/BIMSBbioinfo/scmaui-experiments/blob/main/tutorials/03.%20Single-cell%20Multiomics%20data%20analysis%20based%20on%20scMaui%20latents.ipynb) and [Tutorial 04](https://github.com/BIMSBbioinfo/scmaui-experiments/blob/main/tutorials/04.%20scMaui%20latent%20analysis.ipynb).

In [29]:
dataset = SCDataset(adatas, losses=["negbinom", "binary"], # Binary loss for scBS-seq
                    union=True, adversarial=["embryo"],
                    conditional=["embryo"])#

params = init_model_params()
params.update({'losses': dataset.losses})
params['kl_weight'] = 0.3
modalities = dataset.modalities()
params['input_modality'] = modalities[0]
params['output_modality'] = modalities[1]
params["nlatent"] = 30
params.update(dataset.adversarial_config())
params.update(dataset.conditional_config())
params

# Create a model
ensemble = EnsembleVAE(params=params, ensemble_size=1)
os.environ["CUDA_VISIBLE_DEVICES"]="-1" # This line is for avoiding tensorflow models to be allocated to GPUs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # This line disables Tensorflow debugging information
train_loss = ensemble.fit(dataset, epochs=100, 
                          learning_rate=0.001, batch_size=128)

using vae
Run model 1
Epoch 1/100
7/7 - 1s - adv: 1561.8699 - kl: 510.3268 - recon: 16871.5737 - loss: 15820.0305 - val_adv: 225.7889 - val_kl: 920.3942 - val_recon: 15197.1201 - val_loss: 15891.7256
Epoch 2/100
7/7 - 0s - adv: 1727.3707 - kl: 951.8890 - recon: 14547.2672 - loss: 13771.7860 - val_adv: 152.4576 - val_kl: 795.0279 - val_recon: 13257.5752 - val_loss: 13900.1455
Epoch 3/100
7/7 - 0s - adv: 1479.9218 - kl: 615.3281 - recon: 13466.6271 - loss: 12602.0334 - val_adv: 151.0450 - val_kl: 352.7277 - val_recon: 12816.9883 - val_loss: 13018.6709
Epoch 4/100
7/7 - 0s - adv: 1439.6411 - kl: 300.7198 - recon: 13198.2538 - loss: 12059.3324 - val_adv: 140.4270 - val_kl: 211.2634 - val_recon: 12629.2246 - val_loss: 12700.0615
Epoch 5/100
7/7 - 1s - adv: 1477.0756 - kl: 209.7215 - recon: 13146.9377 - loss: 11879.5834 - val_adv: 158.3611 - val_kl: 155.2321 - val_recon: 12542.0137 - val_loss: 12538.8848
Epoch 6/100
7/7 - 1s - adv: 1414.2125 - kl: 173.7845 - recon: 12988.3706 - loss: 11747.9